In [1]:
import cv2
import numpy as np

In [2]:
net=cv2.dnn.readNet( "yolov3.weights","yolov32.cfg",)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
classes=[]
with open("coco1.names","r") as f:
    classes=[line.strip() for line in f.readlines()]

In [3]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0]-1]for i in net.getUnconnectedOutLayers()]
colors= np.random.uniform(0,255,size=(len(classes),3))

In [4]:
cap=cv2.VideoCapture("vid_short.mp4")
center_x=0.0
center_y=0.0
font= cv2.FONT_HERSHEY_PLAIN
while True:
    _,frame= cap.read()
    height,width,channels=frame.shape
    blob=cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False)
    
    net.setInput(blob)
    outs= net.forward(outputlayers)
    
    class_ids=[]
    confidences=[]
    boxes= []
    for out in outs:
        for detection in out:
            scores= detection[5:]
            class_id =np.argmax(scores)
            confidence=scores[class_id]
            if confidence > 0.1 and class_id == 0:
                center_x=int(detection[0]*width)
                center_y=int(detection[1]*height)
                w=int(detection[2]*width)
                h=int(detection[3]*height)
                x=int(center_x -w/2)
                y= int(center_y - h/2)
                
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)
    
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h= boxes[i]
            x_c=int(x+w/2)
            y_c=int(y+h/2)
            label=str(classes[class_ids[i]])
            confidence=confidences[i]
            color=colors[class_ids[i]]
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            #cv2.polylines(frame, [penta], True, (255,120,255),2)
            #cv2.circle(result,(x_c, y_c), 1, (0,255,0), 4)
            #cv2.circle(blankScreen,(x_c, y_c), 1, (0,255,0), 4)
            cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
            #cv2.circle(frame, (center_x,center_y), 1, (2), thickness)
            
    cv2.imshow("Image",frame)
    #cv2.imshow('frame12', blankScreen)
    key=cv2.waitKey(1)
    if key == 27:
        break;
        
cap.release()
cv2.destroyAllWindows()

In [1]:
from scipy.spatial import distance
diff= distance.euclidean([406, 652], [417, 717])

In [2]:
diff

65.92419889539804

In [3]:
diff2= distance.euclidean((312, 183), (372, 269))

In [4]:
diff2

104.86181383134664